# Soluzione per la Maze Challenge
# Studenti: *Antonio Strippoli / Luca Moroni*

## Quest 1-2-3
La prima Quest richiedeva l'esplorazione del labirinto sfruttando il file `mazeClient.py` con delle utility pronte messe a disposizione. Si noti che dalla risoluzione della Quest 1 derivano facilmente la risoluzione della Quest 2 e 3 (che riguardano semplicemente la raccolta di informazioni durante l'esplorazione).

### Analisi
Richiedendo l'output del comando *GET_STATE*, si ottiene qualcosa del genere:

`{'userX': 14, 'userY': 12, 'userVal': 32, 'Neighbors': [{'x': 14, 'y': 13, 'val': 71}]}`

Analizzandolo, si è capito che:
* userX: indice di riga;
* userY: indice di colonna;
* userVal: rappresentazione numerica del colore della cella;
* Neighbors: lista delle celle circostanti.

Si è notato inoltre che *Neighbors* contiene anche le celle diagonalmente vicine, che non sono però raggiungibile mediante un comando di quelli proposti e che andranno perciò tralasciate.

### Progettazione
In base all'analisi fatta, il labirinto è facilmente trattabile come un grafo non orientato. Tra gli algoritmi basilari e più conosciuti per l'esplorazione, ce ne sono 2:
* Esplorazione in ampiezza (BFS): va scartata perché con la lista dei comandi resa disponibile non è possibile spostarsi in un solo passaggio a degli indici stabiliti;
* Esplorazione in profondità (DFS): fattibile, poiché ad ogni passaggio ci si sposta unicamente in celle vicine.

Per cui si è scelto di esplorare il labirinto scrivendo una funzione DFS opportunatamente modificata. Si è implementata una versione ricorsiva per maggiore chiarezza e compattezza del codice (i labirinti generati sono comunque molto piccoli, per cui il rischio di esaurire la memoria è praticamente nullo).

Una versione iterativa dell'algoritmo è stata inizialmente pensata e scritta tramite pseudocodice, che riportiamo di seguito per completezza e future migliorie:
```
- Scegli un vicino NON visitato:
    - Vai dal vicino
- Altrimenti (nessun vicino ok):
    - Controlla se puoi andare back
        - Vai back
    - Altrimenti (sono tornato all'origine):
        - Termina
```
        
### Codice
Si è scelto di creare una classe `Maze` contenente tutti i metodi per l'esplorazione del labirinto e conseguente raccolta dati.
Per esplorare il labirinto, basta creare un oggetto di tipo `Maze`. Verrà richiamato il metodo `__init__()` che inizializzerà le variabili dove verranno salvate le statistiche (in vari formati per comodità), esplorerà il primo nodo e chiamerà la funzione `dfs_visit()` per esplorare il resto del labirinto a partire dal primo nodo. Nello specifico le variabili per la raccolta delle statistiche sono:
* **visited**: una lista che contiene i nodi esattamente come vengono restituiti dal server (utili per il plotting della mappa);
* **colors_x** e **colors_y**: dei dizionari che contengono per ogni x e per ogni y le occorrenze di ogni colore (utile per il plotting della distribuzione dei colori rispetto alle x e alle y;
* **colors_count**: un dizionario che contiene il numero di occorrenze di ogni colore (utile per le Quest 2-3).

In [ ]:
from mazeClient import send_command
from mazeClient import Commands as command
import json
import pickle
from time import sleep

class Maze():
    """
    Class that contains methods to solve the maze
    """
    def __init__(self):
        # Initialize variables used to collect data from maze
        # visited = map representation
        # colors_xy = distribution of colors on x,y axes
        # colors_count = count of each color present in the map
        self.visited = []
        self.colors_x = {}
        self.colors_y = {}
        self.colors_count = {
            'red':   0,
            'green': 0,
            'blue':  0,
            'white': 0
        }

        # Initialize map to better manage colors
        self.c_map = {
            82: 'red',
            71: 'green',
            66: 'blue',
            32: 'white'
        }

        # Visit the root (starting position)
        curr_node = self.get_dict(send_command(command.GET_STATE))
        self.visited.append({
            'x': curr_node['userX'],
            'y': curr_node['userY'],
            'val': curr_node['userVal']
        })

        # Explore the maze
        self.dfs_visit(curr_node, command.GET_STATE)


    def get_dict(self, data: bytes):
        """
        Parse data and returns a dictionary (more usable)
        """
        return json.loads(data.decode('ascii'))


    def get_inverse_command(self, cmd: "mazeClient.Commands"):
        """
        Returns the "Go Back" command
        """
        cmd_map = {
            command.MOVE_LEFT:  command.MOVE_RIGHT,
            command.MOVE_RIGHT: command.MOVE_LEFT,
            command.MOVE_UP:    command.MOVE_DOWN,
            command.MOVE_DOWN:  command.MOVE_UP,
            command.GET_STATE:  command.GET_STATE
        }
        return cmd_map[cmd]


    def get_command_from_pos(self, org: dict, dst: dict) -> "mazeClient.Commands":
        """
        Return command to let you move from org to dst
        """
        diff_x = org['userX'] - dst['x']
        diff_y = org['userY'] - dst['y']

        if diff_x == 1:
            return command.MOVE_DOWN
        elif diff_x == -1:
            return command.MOVE_UP
        elif diff_y == 1:
            return command.MOVE_RIGHT
        elif diff_y == -1:
            return command.MOVE_LEFT
        return command.GET_STATE  # Bad usage


    def get_reachable_neighbors(self, v: dict):
        """
        Returns valid neighbors (excludes the diagonal ones)
        """
        tmp = []
        for el in v["Neighbors"]:
            if (el["x"] - v["userX"] == 0) or (el["y"] - v["userY"] == 0):
                tmp.append(el)
        return tmp


    def visit_node(self, node: dict):
        """
        Visit a node and save informations about it
        """
        # Extract data from node
        node_x = node['x']
        node_y = node['y']
        node_color = self.c_map[node['val']]

        # Save informations
        self.visited.append(node)
        self.colors_count[node_color] += 1
        self.colors_x.setdefault(node_x, {
            'red':   0,
            'green': 0,
            'blue':  0,
            'white': 0
        })[node_color] += 1
        self.colors_y.setdefault(node_y, {
            'red':   0,
            'green': 0,
            'blue':  0,
            'white': 0
        })[node_color] += 1


    def dfs_visit(self, v: dict, last_cmd: str):
        """
        DFS Algorithm to explore the maze
        """
        for u in self.get_reachable_neighbors(v):
            if u not in self.visited:
                # Visit the neighbor
                self.visit_node(u)

                # Move to neighbor
                cmd = self.get_command_from_pos(v, u)
                u = self.get_dict(send_command(cmd))
                #sleep(0.5)

                # Visit from that neighbor
                self.dfs_visit(u, cmd)

        # Move back, no more valid neighbors
        send_command(self.get_inverse_command(last_cmd))
        #sleep(0.5)

## Quest 4, 5 (and extra 3); Advanced Quest 1, 2
Per risolvere le Quest 4, 5 si è utilizzata la libreria **matplotlib**, utilizzando i dati raccolti durante l'esplorazione del labirinto. Si è deciso inoltre di fare un passo in più rispetto a quanto richiesto della Quest 3, plottando anche un istogramma relativo alla distribuzione dei colori in tutta la mappa.
Si noti che le funzioni presentate di seguito sono state poste in un file a parte (`stats.py`) per fare ordine.

In [ ]:
# Import for plotting operations
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

### `plot_map()`
Visualizza il plot di una matrice che rappresenta la mappa del labirinto. Si è scelto di utilizzare la funzione di matplotlib [mathshow()](https://matplotlib.org/3.2.1/api/_as_gen/matplotlib.pyplot.matshow.html).

Per rappresentare con dei colori personalizzati la matrice, si è effettuato un mapping dei colori a partire dagli interi restituiti dal server, per poi far effettuare un secondo mapping a matplotlib per l'assegnazione vera e propria dei colori.

Si noti che il calcolo dei valori minimi e massimi è stato scritto in maniera poco efficiente per favorire la leggibilità e compattezza del codice.

In [ ]:
def plot_map(visited: list):
    """
    Plots the map constructing a matrix.
    Remember that x = row index, y = column index.
    """
    # Colors mapping for matshow library method
    colors_map = {82: 1, 71: 2, 66: 3, 32: 4}
    cmap = ListedColormap(['k', 'r', 'g', 'b', 'w'])
    
    # Get the coordinates max and min
    x_min = min(visited, key=lambda el:el['x'])['x']
    x_max = max(visited, key=lambda el:el['x'])['x']
    y_min = min(visited, key=lambda el:el['y'])['y']
    y_max = max(visited, key=lambda el:el['y'])['y']

    matrix_plt = np.zeros((x_max - x_min + 1, y_max - y_min + 1))
    for el in visited:
        matrix_plt[
            x_max - el["x"],
            y_max - el["y"]
        ] = colors_map[el["val"]]

    # Plotting the matrix
    plt.matshow(matrix_plt, cmap=cmap)
    plt.suptitle('Maze representation')
    plt.xticks(range(0, y_max-y_min+1, 2), range(y_max, y_min-1, -2))
    plt.yticks(range(0, x_max-x_min+1, 2), range(x_max, x_min-1, -2))
    plt.show()

### `plot_colors_dist()`
Visualizza un istogramma rappresentante la distribuzione dei colori nella mappa. Si è scelto di utilizzare la funzione di matplotlib [bar()](https://matplotlib.org/3.2.1/api/_as_gen/matplotlib.pyplot.bar.html).

Si noti che viene anche calcolato il numero di celle totali presenti nella mappa, come richiesto dalla Quest 2.

In [ ]:
def plot_colors_dist(nodes_count: dict):
    """
    Additional plot that shows colors distribution in the map (not requested by any quest) 
    """
    # Prepare data
    names = list(nodes_count.keys())
    values = list(nodes_count.values())
    colors = ['0.5', 'r', 'g', 'b']
    total_cells = sum(nodes_count.values())

    # Plot
    fig, ax = plt.subplots()
    fig.suptitle(f"Colors distribution (total cells: {total_cells})")
    rects = ax.bar(names, values, color=colors, align='center')

    # Attach a text label above each bar in rects, displaying its height
    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 1),  # 1 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')

    plt.show()

### `plot_colors_xy_dist()`
Visualizza il plot delle distribuzioni dei colori rispetto alle x ed alle y. Si è scelto di utilizzare la funzione di matplotlib [bar()](https://matplotlib.org/3.2.1/api/_as_gen/matplotlib.pyplot.bar.html).

Sono state create delle procedure interne per evitare ridondanza nel codice e aumentarne la leggibilità. Nello specifico:
* `preprocess_data_hist()`: prepara i dati per essere utilizzati da matplotlib;
* `plot_axes()`: visualizza con un grafico a barre la distribuzione dei colori. Si noti che in particolare questa funzione viene richiamata 2 volte, per permettere il confronto delle distribuzioni tra il vecchio labirinto ed il corrente.

In [ ]:
def plot_colors_xy_dist(maze: "Maze", past_maze: "Maze"):
    """
    Plot a grouped bar chart that represents colors distribution on x and y
    """
    def preprocess_data_hist(colors_xy: tuple):
        """
        Internal function that preprocess data for matplotlib
        """
        colors_x, colors_y = colors_xy

        x_label = []
        x_red   = []
        x_green = []
        x_blue  = []
        x_white = []
        
        y_label = []
        y_red   = []
        y_green = []
        y_blue  = []
        y_white = []

        for key in sorted(colors_x):
            x_label.append(key)
            x_red.append(colors_x[key]["red"])
            x_green.append(colors_x[key]["green"])
            x_blue.append(colors_x[key]["blue"])
            x_white.append(colors_x[key]["white"])

        for key in sorted(colors_y):
            y_label.append(key)
            y_red.append(colors_y[key]["red"])
            y_green.append(colors_y[key]["green"])
            y_blue.append(colors_y[key]["blue"])
            y_white.append(colors_y[key]["white"])

        return x_label, x_red, x_green, x_blue, x_white, y_label, y_red, y_green, y_blue, y_white


    def plot_axes(ax1, ax2, colors_xy, label="", width=0.2):
        """
        Internal function that plots colors distribution on given axes
        Width is used to determine the width of each bar
        """
        # Preprocess data
        x_label, x_red, x_green, x_blue, x_white, y_label, y_red, y_green, y_blue, y_white = preprocess_data_hist(colors_xy)

        # Prepare bars for colors x distribution
        x = np.arange(len(x_label))
        ax1.bar(x - 3*width/2, x_red, width, label='Red', color="red")
        ax1.bar(x - width/2, x_green, width, label='Green', color="green")
        ax1.bar(x + width/2, x_blue, width, label='Blue', color="blue")
        ax1.bar(x + 3*width/2, x_white, width, label='White', color="grey")

        # Add some text for labels, title and custom x-axis tick labels, etc.
        ax1.set_ylabel('Frequency')
        ax1.set_title(label + "X (rows)")
        ax1.set_xticks(x)
        ax1.set_xticklabels(x_label)

        # Prepare bars for colors y distribution
        y = np.arange(len(y_label))  # the label locations
        ax2.bar(y - 3*width/2, y_red, width, label='Red', color="red")
        ax2.bar(y - width/2, y_green, width, label='Green', color="green")
        ax2.bar(y + width/2, y_blue, width, label='Blue', color="blue")
        ax2.bar(y + 3*width/2, y_white, width, label='White', color="grey")

        # Add some text for labels, title and custom x-axis tick labels, etc.
        ax2.set_ylabel('Frequency')
        ax2.set_title(label + "Y (cols)")
        ax2.set_xticks(y)
        ax2.set_xticklabels(y_label)


    # Create figure, axes and then plot
    if past_maze:
        fig, ((ax11, ax12), (ax21, ax22)) = plt.subplots(2,2)
        plot_axes(ax21, ax22, (past_maze.colors_x, past_maze.colors_y), label="PAST maze color distribution on ")
    else:
        fig, (ax11, ax12) = plt.subplots(1,2)
    
    fig.set_size_inches(12, 8)
    plot_axes(ax11, ax12, (maze.colors_x, maze.colors_y), label="CURRENT maze color distribution on ")
    plt.show()

### Main
Di seguito il codice principale del programma. Si noti che per risolvere l'Advanced Quest 2, si è scelto di salvare l'oggetto *maze* in un file **pickle**. Questo è utilizzato per fare una comparazione tra la distribuzione dei colori del labirinto della precedente esecuzione e di quello della corrente esecuzione.

In [ ]:
# Explore the Maze (Quests 1-2-3)
maze = Maze()

# Get data of past map (if they exist)
try:
    with open('past_maze.pickle', 'rb') as f:
        past_maze = pickle.load(f)
except FileNotFoundError:
    past_maze = None

# Plot statistics of the maze (Quests 3-4-5, Advanced Quest 2)
plot_colors_dist(maze.colors_count)
plot_colors_xy_dist(maze, past_maze)
plot_map(maze.visited)

# Save current map (Part of Advanced Quest 2)
with open('past_maze.pickle', 'wb') as f:
    pickle.dump(maze, f)

## Advanced Quest 3

Per l'ultima quest, si è creato un file a parte (`client_controller.py`). Si è utilizzata la libreria **pynput** per l'acquisizione dell'input.

Si è scelto di usare i tasti 'WASD' per muoversi nel labirinto, in quanto le frecce direzionali vengono già prese in input dal server se la finestra è attiva, rendendo totalmente inutile implementarne il supporto.

Si noti inoltre che si è anche scelto di rendere disponibile la pressione del tasto 'E' per ottenere le informazioni sul nodo corrente.

In [ ]:
# -*- coding: utf-8 -*-
from mazeClient import Commands as command
from mazeClient import send_command
from pynput import keyboard


def on_press(key):
    """
    Listen for input and move if any of 'WASD' is pressed
    Exit if any other key is pressed
    """
    # Not a valid key pressed?
    if not hasattr(key, 'char'):
        return False
    if not key.char in keycode_map:
        return False
    
    # Map keycode to action and execute action
    action = keycode_map[key.char]
    res = send_command(action)

    # Just some print to let user have some feedback
    if action == command.GET_STATE:
        print(res)
    else:
        print(action)


if __name__ == "__main__":
    # Initialize mapping variable
    keycode_map = {
        'w': command.MOVE_UP,
        'a': command.MOVE_LEFT,
        's': command.MOVE_DOWN,
        'd': command.MOVE_RIGHT,
        'e': command.GET_STATE
    }
    
    print("INSTRUCTIONS:\n\tWASD -> Move around the maze;\n\tE -> GET STATE;\n\tAny other Key: QUIT")
    # Collect events until released
    with keyboard.Listener(on_press=on_press) as listener:
        listener.join()